In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install chromadb
%pip install llama-index-vector-stores-chroma
#llama-index-embeddings-huggingface vs llama-index c nlk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s

In [2]:
import json
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

In [3]:
#vars
model_embedding = "intfloat/multilingual-e5-large"
file_with_docs = '/content/test.json' #ссылка на файлы джейсон
name_collection = 'sites' # название коллекции, куда сохранить chroma

In [4]:
#парсинг json
def create_document_from_json(url,detail,user=1):
  content = details["content"]
  date = details["date"]
  return Document(text=content, metadata={"url": url, "date": date , "user": user})

In [5]:
# построние хранилища

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection(name_collection)

# define embedding function
embed_model = HuggingFaceEmbedding(model_name=model_embedding)
Settings.embed_model = embed_model

with open(file_with_docs, 'r', encoding='utf-8') as file:
    data = json.load(file)

documents = []
for url, details in data.items():
    documents.append(create_document_from_json(url ,details))

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [8]:
def print_chunks(query):
  chunks = retriever.retrieve(query)

  for chunk in chunks:
    print(len(chunk.text))
    print(chunk.metadata['url'])
    print(chunk.text)
    print('--')

In [9]:
# поиск примеров
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
query = "пиццы заказать быстрая доставка "

# создания поисковика
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)
print_chunks(query)



1444
https://www.russianfood.com/recipes/bytype/?fid=30
Как готовить пиццу и тесто к ней? Приготовление пиццы, конечно же, следует начинать с готовки теста. Тесто для пиццы бывает нескольких видов. На нашем сайте вы можете найти следующие рецепты теста для пиццы: тесто для пиццы на кефире, дрожжевое тесто для пиццы, тесто для пиццы без дрожжей, пицца из слоеного теста, тесто для пиццы тонкое, быстрое тесто для пиццы.

Как приготовить пиццу дома? Приготовить пиццу в домашних условиях легче всего по таким распространенным рецептам как: пицца с колбасой, пицца с колбасой и сыром, пицца с грибами, куриная пицца (рецепты с фото покажут, как выглядит готовое блюдо). Домашние рецепты пиццы имеют множество вариантов приготовления. Рецепт пиццы в домашних условиях может меняться в зависимости от того, какие продукты есть у вас под рукой. Как сделать пиццу по-настоящему вкусной? Вы можете экспериментировать с различными продуктами, добавляя что-то новое и исключая то, что не пришлось по вкусу. Д

Добавление нового элемента

In [10]:
print_chunks('Лошади ')

1777
https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82
Ишкушштвенный интеллект
А. Соловьёв. Искусственный интеллект давно родился Архивная копия от 7 сентября 2019 на Wayback Machine


 
  Словари и энциклопедии
	
Большая датскаяБольшая китайскаяБольшая норвежскаяБольшая норвежскаяБольшая российская (старая версия)КругосветЛаруссаНациональная (Латвия)Стэнфордская философскаяBritannica (онлайн)De AgostiniPWNTreccani


В библиографических каталогах
Искусственный интеллект
Категория: Искусственный интеллект
Навигация
Вы не представились системе
Обсуждение
Вклад
Создать учётную запись
Войти
Статья
Обсуждение
Читать
Править
Править код
История
Поиск
Заглавная страница
Содержание
Избранные статьи
Случайная статья
Текущие события
Пожертвовать
Участие
Сообщить об ошибке
Как править статьи
Сообщество
Форум
Свежие правки
Новые страницы
Справка
Инструменты
Ссылки сюда
Связанные правки

In [11]:
#добавление нового элемента
index.insert(Document(text='розовые пони скачут на лугу и кричат ауу', metadata={"url": url,  "user": -1}))

In [14]:
print_chunks('лошади')

40
https://zoon.ru/nn/gorod-arzamas/fitness/
розовые пони скачут на лугу и кричат ауу
--
238
https://www.google.com/search?q=%D1%81%D0%BF%D0%BE%D1%80%D1%82+%D0%BA%D0%BB%D1%83%D0%B1+%D0%B0%D1%80%D0%B7%D0%B0%D0%BC%D0%B0%D1%81&sca_esv=31fc9eb8bb598b81&rlz=1C1GCEA_enRU1046RU1046&ei=XHIXZ6rcLeCFxc8Pto6tsAY&oq=%D1%81%D0%BF%D0%BE%D1%80%D1%82+%D0%BA%D0%BB%D1%83%D0%B1+%D0%B0%D1%80%D0%B7%D0%B0%D0%BC%D0%B0&gs_lp=Egxnd3Mtd2l6LXNlcnAiINGB0L_QvtGA0YIg0LrQu9GD0LEg0LDRgNC30LDQvNCwKgIIADIHECEYoAEYCjIHECEYoAEYCjIFECEYnwUyBRAhGJ8FMgUQIRifBTIFECEYnwUyBRAhGJ8FMgUQIRifBTIFECEYnwUyBRAhGJ8FSOB6UK5HWNtzcAN4AZABAJgBqQKgAZMaqgEGMC4xMy40uAEDyAEA-AEBmAIUoALrG6gCFMICExAAGIAEGEMYtAIYigUY6gLYAQHCAhMQLhiABBhDGLQCGIoFGOoC2AEBwgIdEAAYgAQYtAIY1AMY5QIYtwMYigUY6gIYigPYAQHCAhYQABgDGLQCGOUCGOoCGIwDGI8B2AECwgIWEC4YAxi0AhjlAhjqAhiMAxiPAdgBAsICHBAuGNEDGAMYtAIY5QIYxwEY6gIYjAMYjwHYAQLCAgsQABiABBixAxiDAcICDhAuGIAEGLEDGIMBGIoFwgIFEC4YgATCAhEQLhiABBixAxjRAxiDARjHAcICBRAAGIAEwgIKEC4YgAQYQxiKBcICDRAuGIAEGEMY1AIYigXCAhAQLhiABBixAxhDGIM